In [1]:
from cobra.io import read_sbml_model
from pathlib import Path
import os
import cobra

cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') # set cwd

# LOAD RPE-PR MODEL
mod_RPE_PR = read_sbml_model(Path().cwd() / 'mod_RPE_PR.xml')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


In [2]:
# SET EXCHANGE BOUNDS
# 'id' col with IDs and an 'upper_bound' col with bounds
import pandas as pd
bounds = pd.read_excel(Path().cwd() / 'rxn_bounds' / 'R3D301_EX_rxns.xlsx')
bounds = bounds[bounds['upper_bound']>0] # select entries with non-zero upper bounds

# open RPE exchange reactions
for i in range(len(bounds)): 
    mod_RPE_PR.reactions.get_by_id(bounds['id'][i]+'_RPE').bounds = (-bounds['upper_bound'][i],1000)   
    
# close PR exchange reactions  
rxns = [r for r in mod_RPE_PR.reactions if '_PR' in r.id if len(r.products) == 0]
for r in rxns:
    r.bounds=(0,0)

In [3]:
# MAKE DF WITH MET / RXN INFO TO ADD FBA RESULTS TO
def get_annotation_ids(x, annotation_keys):
    annotation = []
    for k in annotation_keys:
        if k in x.annotation.keys():
            annotation.append(x.annotation[k])
        else:
            annotation.append('')
    return annotation

# RXNS
l = [list(r.annotation.keys()) for r in  mod_RPE_PR.reactions]
annotation_keys = list(set([item for sublist in l for item in sublist]))
rxns = pd.DataFrame([[r.id,r.lower_bound,r.upper_bound]+get_annotation_ids(r,annotation_keys)+\
                     [r.name,r.subsystem,r.compartments,\
                      r.reaction,r.build_reaction_string(use_metabolite_names = True),r.gpr]\
                      for r in mod_RPE_PR.reactions],index=[r.id for r in mod_RPE_PR.reactions],\
                    columns=['Human1.reaction','lb','ub',]+annotation_keys+\
                    ['name','subsystem','compartments','reaction (IDs)','reaction (names)','GPR'])

In [4]:
# OBJECTIVE 
# set maximizing the ATP demand in PR as objective
mod_RPE_PR.objective = mod_RPE_PR.reactions.get_by_id('MAR03964_PR').flux_expression
mod_RPE_PR.reactions.get_by_id('MAR03964_PR')

Reaction identifier,MAR03964_PR
Name,ATP phosphohydrolase
Memory address,0x1ba79892050
Stoichiometry,MAM01371c_PR + MAM02040c_PR --> MAM01285c_PR + MAM02039c_PR + MAM02751c_PR ATP + H2O --> ADP + H+ + Pi
GPR,
Lower bound,0.0
Upper bound,1000.0


In [5]:
# FBA
solution = mod_RPE_PR.optimize()

In [6]:
from cobra.flux_analysis import flux_variability_analysis
fva = flux_variability_analysis(mod_RPE_PR,mod_RPE_PR.reactions[0:2], loopless=True)

In [7]:
fva

,minimum,maximum
MAR03905_RPE,0.0,0.0
MAR03907_RPE,0.0,0.0


In [8]:
# INPUT / OUTPUT behaviour
s = mod_RPE_PR.summary()

In [9]:
mod_RPE_PR

Name,RPE_PR
Memory address,1ba6fe34dc0
Number of metabolites,10436
Number of reactions,14282
Number of genes,2268
Number of groups,142
Objective expression,1.0*MAR03964_PR - 1.0*MAR03964_PR_reverse_e8402
Compartments,"c_RPE, r_RPE, e_RPE, m_RPE, x_RPE, g_RPE, l_RPE, n_RPE, i_RPE, e_RPE_PR, c_PR, x_PR, m_PR, g_PR, r_PR, l_PR, i_PR"


In [10]:
s_fva = mod_RPE_PR.summary(fva=.90)

In [12]:
# FBA for 
s = pd.DataFrame(index=[r.id for r in mod_RPE_PR.reactions])
import warnings
for fixed_flux in range(101):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        mod_RPE_PR.reactions.get_by_id('MAR03964_RPE').bounds = (fixed_flux, fixed_flux)
        s_i = mod_RPE_PR.optimize()
        if s_i.status == 'optimal':
            s_i = s_i.to_frame()
            s_i.columns = s_i.columns + '_' + str(fixed_flux)  
            s = pd.merge(s,s_i,left_index=True, right_index=True) 

# reset ATP demand bounds in RPE
mod_RPE_PR.reactions.get_by_id('MAR03964_RPE').bounds = (0, 1000)

In [13]:
# select non-zero FBA fluxes
s_flux = s.loc[:,[c for c in list(s.columns) if 'fluxes' in c]]
s_flux = s_flux.loc[(s_flux!=0).any(axis=1)]
s_flux.to_clipboard(excel=True, sep=None)

# RXNS
rxns = pd.DataFrame([[r.id,r.id.split('_')[1].replace('eRPE','RPE_PR_interface'),r.lower_bound,r.upper_bound, r.name,r.subsystem,\
                      r.reaction,r.build_reaction_string(use_metabolite_names = True),r.gpr]\
                      for r in mod_RPE_PR.reactions],index=[r.id for r in mod_RPE_PR.reactions],\
                    columns=['Human1.reaction','cell','lb','ub','name','subsystem',\
                             'reaction (IDs)','reaction (names)','GPR'])

fluxes_fba = pd.merge(rxns.copy(),s_flux,left_index=True, right_index=True) 
fluxes_fba.to_clipboard(excel=True, sep=None)
fluxes_fba

,Human1.reaction,cell,lb,ub,name,subsystem,reaction (IDs),reaction (names),GPR,fluxes_0,...,fluxes_74,fluxes_75,fluxes_76,fluxes_77,fluxes_78,fluxes_79,fluxes_80,fluxes_81,fluxes_82,fluxes_83
MAR04388_RPE,MAR04388_RPE,RPE,-1000.0,1000.0,(S)-Lactate:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM02039c_RPE + MAM02553c_RPE + MAM02819c_RPE ...,H+ + NADH + pyruvate <=> L-lactate + NAD+,ENSG00000111716 or ENSG00000134333 or ENSG0000...,-0.388718,...,60.513077,61.513077,62.513077,63.513077,64.513077,65.513077,66.513077,67.513077,68.513077,69.513077
MAR04301_RPE,MAR04301_RPE,RPE,0.0,1000.0,UTP:D-fructose-6-phosphate 1-phosphotransferase,Glycolysis / Gluconeogenesis,MAM01845c_RPE + MAM03130c_RPE --> MAM01841c_RP...,"fructose-6-phosphate + UTP --> fructose-1,6-bi...",ENSG00000067057 or ENSG00000141959 or ENSG0000...,0.000000,...,35.392436,35.892436,36.392436,36.892436,37.392436,37.892436,38.392436,38.892436,39.392436,39.892436
MAR04363_RPE,MAR04363_RPE,RPE,-1000.0,1000.0,2-phospho-D-glycerate hydro-lyase (phosphoenol...,Glycolysis / Gluconeogenesis,MAM00674c_RPE <=> MAM02040c_RPE + MAM02696c_RPE,2-phospho-D-glycerate <=> H2O + PEP,ENSG00000074800 or ENSG00000108515 or ENSG0000...,0.000000,...,71.764359,72.764359,73.764359,74.764359,75.764359,76.764359,77.764359,78.764359,79.764359,80.764359
MAR04365_RPE,MAR04365_RPE,RPE,-1000.0,1000.0,"D-phosphoglycerate 2,3-phosphomutase",Glycolysis / Gluconeogenesis,MAM00674c_RPE <=> MAM00913c_RPE,2-phospho-D-glycerate <=> 3-phospho-D-glycerate,ENSG00000164708 or ENSG00000171314 or ENSG0000...,0.000000,...,-70.784872,-71.784872,-72.784872,-73.784872,-74.784872,-75.784872,-76.784872,-77.784872,-78.784872,-79.784872
MAR04368_RPE,MAR04368_RPE,RPE,-1000.0,1000.0,ATP:3-phospho-D-glycerate 1-phosphotransferase,Glycolysis / Gluconeogenesis,MAM00247c_RPE + MAM01285c_RPE <=> MAM00913c_RP...,"1,3-bisphospho-D-glycerate + ADP <=> 3-phospho...",ENSG00000096006 or ENSG00000102144 or ENSG0000...,0.000000,...,70.784872,71.784872,72.784872,73.784872,74.784872,75.784872,76.784872,77.784872,78.784872,79.784872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAR10366_PR_RPE,MAR10366_PR_RPE,PR,-1000.0,1000.0,"Transport of Methylmalonate, Antiport with Phs...",Transport reactions,MAM02479c_PR + MAM02751e_RPE_PR <=> MAM02479e_...,methylmalonate + Pi <=> methylmalonate + Pi,,0.281923,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MAR10392_PR_RPE,MAR10392_PR_RPE,PR,-1000.0,1000.0,"Transport of Oxaloacetate, Proton Symport",Transport reactions,MAM02039c_PR + MAM02633c_PR <=> MAM02039e_RPE_...,H+ + OAA <=> H+ + OAA,,-2.087436,...,-2.651282,-2.651282,-2.651282,-2.651282,-2.651282,-2.651282,-2.651282,-2.651282,-2.651282,-2.651282
MAR10483_PR_RPE,MAR10483_PR_RPE,PR,-1000.0,1000.0,Transport of Retinal,Transport reactions,MAM02832c_PR <=> MAM02832e_RPE_PR,retinal <=> retinal,,2.651282,...,2.651282,2.651282,2.651282,2.651282,2.651282,2.651282,2.651282,2.651282,2.651282,2.651282
MAR10487_PR_RPE,MAR10487_PR_RPE,PR,-1000.0,1000.0,Transport of Prolylglycine,Transport reactions,MAM03896c_PR <=> MAM03896e_RPE_PR,L-Prolinylglycine <=> L-Prolinylglycine,,0.903333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
non_zero_flux_rxns = [mod_RPE_PR.reactions.get_by_id(r) for r in list(fluxes_fba.index)]

In [ ]:
from cobra.flux_analysis import flux_variability_analysis
fva = flux_variability_analysis(mod_RPE_PR,non_zero_flux_rxns, loopless=True)

In [ ]:
fva.to_clipboard(excel=True, sep=None)

In [ ]:
# FBA loopless 
from cobra.flux_analysis.loopless import add_loopless, loopless_solution

s = pd.DataFrame(index=[r.id for r in mod_RPE_PR.reactions])
import warnings
for fixed_flux in range(101):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        mod_RPE_PR.reactions.get_by_id('MAR03964_RPE').bounds = (fixed_flux, fixed_flux)
        s_i = loopless_solution(mod_RPE_PR)
        if s_i.status == 'optimal':
            s_i = s_i.to_frame()
            s_i.columns = s_i.columns + '_' + str(fixed_flux)  
            s = pd.merge(s,s_i,left_index=True, right_index=True) 

# reset ATP demand bounds in RPE
mod_RPE_PR.reactions.get_by_id('MAR03964_RPE').bounds = (0, 1000)

# select non-zero FBA fluxes
s_flux = s.loc[:,[c for c in list(s.columns) if 'fluxes' in c]]
s_flux = s_flux.loc[(s_flux!=0).any(axis=1)]
s_flux.to_clipboard(excel=True, sep=None)

# RXNS
rxns = pd.DataFrame([[r.id,r.id.split('_')[1].replace('eRPE','RPE_PR_interface'),r.lower_bound,r.upper_bound, r.name,r.subsystem,\
                      r.reaction,r.build_reaction_string(use_metabolite_names = True),r.gpr]\
                      for r in mod_RPE_PR.reactions],index=[r.id for r in mod_RPE_PR.reactions],\
                    columns=['Human1.reaction','cell','lb','ub','name','subsystem',\
                             'reaction (IDs)','reaction (names)','GPR'])

fluxes_fba = pd.merge(rxns.copy(),s_flux,left_index=True, right_index=True) 
fluxes_fba.to_clipboard(excel=True, sep=None)
fluxes_fba